# Notebook for Coursera Capstone project

# week 1

In [74]:
# import libs
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [75]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


[Watson Studio share link](https://jp-tok.dataplatform.cloud.ibm.com/analytics/notebooks/v2/1f93ac15-f21b-449c-b6da-ac9ed73e778a/view?access_token=25d992bd79bc2469e5a1a6f3ebb529d4e3e3d7fa53822dd1dae17ff79a45995c)

# week 3

In [76]:
# Setup
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

## Read table from wikipedia.  The post code is contained in  the first 'Postcode' table.

In [77]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url, match='Postcode', header=0)
postcode=dfs[0]
#print(type(postcode))
#print(postcode.shape)
postcode.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Remove Not Assigned Boroughs

In [78]:
# remove Not assigned
postcode_edit=postcode[postcode['Borough']!='Not assigned']
postcode_edit.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Replace NA Neighbourhoods with its Borough

In [79]:
# set Borough for Not assigned Neighbourhood
idx=postcode_edit['Neighbourhood']=='Not assigned'
postcode_edit.loc[idx, 'Neighbourhood']=postcode_edit.loc[idx, 'Borough']
#postcode_edit.set_index('Postcode').reset_index()
postcode_edit[idx]

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


## group and join same postcode and Borough

In [80]:
pcdata = postcode_edit.groupby(['Postcode', 'Borough'], sort=False)['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
pcdata.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## Shape of final dataframe

In [81]:
pcdata.shape

(103, 3)

# week3-part2

## setup geocoder

In [82]:
!pip install geocoder
import geocoder # import geocoder

## Define util function for geocoder

In [83]:
def findlonlat_geocoder(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code)) # ACCESS_DENIED; probably require access token
        # this seem to reliably work, Thanks to Joana Smith - https://www.coursera.org/learn/applied-data-science-capstone/profiles/08f0dfd5c661ddf3f9a548157003dff4
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        print('{}:{}'.format(postal_code, g.latlng))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return(latitude, longitude)

### test

In [91]:
findlonlat_geocoder('M5G')

M5G:[43.65609081300005, -79.38492999999994]


(43.65609081300005, -79.38492999999994)

## Add lonlat of each postal code

In [85]:
data = pcdata['Postcode'].apply(lambda x: pd.Series(findlonlat_geocoder(x)))
data.columns=['Latitude', 'Longitude']
data.head()

M3A:[43.752440000000036, -79.32927072599995]
M4A:[43.730420577000075, -79.31331999999998]
M5A:[43.65512000000007, -79.36263979699999]
M6A:[43.72312500000004, -79.45158914699994]
M7A:[43.66110229800006, -79.39103499999999]
M9A:[43.662242201000026, -79.52837877199994]
M1B:[43.811525000000074, -79.19551721399995]
M3B:[43.74919500000004, -79.36190541699995]
M4B:[43.707535000000064, -79.31177329699995]
M5B:[43.65736301100003, -79.37817999999999]
M6B:[43.70799000000005, -79.44836733199998]
M9B:[43.64969222700006, -79.55394499999994]
M1C:[43.78573000000006, -79.15874999999994]
M3C:[43.72142500000007, -79.34345278999996]
M4C:[43.68966500000005, -79.30716910999996]
M5C:[43.65121000000005, -79.37548057699996]
M6C:[43.692105179000066, -79.43035499999996]
M9C:[43.648573449000025, -79.57824999999997]
M1E:[43.76569000000006, -79.17525603599995]
M4E:[43.67684518300007, -79.29522499999996]
M5E:[43.64516015600003, -79.37367499999993]
M6E:[43.68864000000008, -79.45101525299998]
M1G:[43.76835912100006, -

,Latitude,Longitude
0,43.752440,-79.329271
1,43.730421,-79.313320
2,43.655120,-79.362640
3,43.723125,-79.451589
4,43.661102,-79.391035


## concat geocode

In [86]:
pcdata_new=pd.concat([pcdata, data], axis=1)

In [89]:
pcdata_new.head(30)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035
5,M9A,Etobicoke,Islington Avenue,43.662242,-79.528379
6,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
7,M3B,North York,Don Mills North,43.749195,-79.361905
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.707535,-79.311773
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180
